Installing Necassary Packages

In [1]:
%pip install -q llama-index
%pip install -q openai
%pip install -q transformers
%pip install -q accelerate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q python-dotenv

Note: you may need to restart the kernel to use updated packages.


Importing Libraries and Setting up Environment

In [3]:
import os
import openai
from dotenv import load_dotenv
load_dotenv() 

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

from IPython.display import Markdown, display
from llama_index.llms import openai
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

Loading Data

In [5]:
documents = SimpleDirectoryReader("data").load_data()

Parsing the Documentss into Nodes

In [6]:
from llama_index.core.node_parser import SimpleNodeParser
parser = SimpleNodeParser.from_defaults(chunk_size=1024,chunk_overlap=20)
pdf_nodes = parser.get_nodes_from_documents(documents)


Creating Vector Store

In [7]:
pdf_index=VectorStoreIndex.from_documents(documents)

Creating simple query engine and testing the engine

In [8]:
query_engine = pdf_index.as_query_engine()

In [9]:
response=query_engine.query("When do intermediate admissions open?")
display(Markdown(f"{response}"))

Admissions for intermediate studies start after the Board of Intermediate and Secondary Education releases the matriculation results. Students who have completed their O-levels and received their Cambridge results can also apply, but the requirements for admission may vary for them.

In [10]:
response=query_engine.query("What is the fees for BBA program?")
display(Markdown(f"{response}"))

The fees for the BBA program include a Tuition Fee of Rs. 156,000, Funds of Rs. 24,900, and a Total Annual Fee of Rs. 180,900.

In [11]:
response=query_engine.query("What is the fees for different bachelors programs?")
display(Markdown(f"{response}"))

The fees for different bachelor's programs are as follows:
- For BS/BFA/BD Programs (With Practical): Tuition Fee is Rs. 140,100, Total Annual Fee is Rs. 167,000.
- For BBA/Accounting & Finance/LLB Programs: Tuition Fee is Rs. 156,000, Total Annual Fee is Rs. 180,900.
- For BS/B.Ed Programs (Without Practical): Tuition Fee is Rs. 128,100, Total Annual Fee is Rs. 153,000.

In [12]:
response=query_engine.query("List the core courses in the BSCS programme?")
display(Markdown(f"{response}"))

The core courses in the BSCS program include a wide matrix of interesting courses that aim to provide students with a broad understanding of Computer Science concepts, techniques, and theory. The program offers a balanced mixture of theory and practical experiences at foundation and advanced levels to equip graduates with the ability to make sound professional decisions in the field of Computer Science. Additionally, the curriculum focuses on developing critical professional thinking and intuition in students.

Storing the index to memory

In [13]:
# Persisting to disk
pdf_index.storage_context.persist(persist_dir="./KBot Storage")

# Loading from disk
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./KBot Storage")
index = load_index_from_storage(storage_context)

In [14]:
import nest_asyncio
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import ServiceContext

nest_asyncio.apply()

# We are using the LlamaDebugHandler to print the trace of the sub questions captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

service_context = ServiceContext.from_defaults(
    callback_manager=callback_manager
)

C:\Users\PMLS\AppData\Local\Temp\ipykernel_6200\3467059896.py:13: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [15]:
vector_query_engine = VectorStoreIndex.from_documents(
    documents,
    use_async=True,service_context=service_context
).as_query_engine()

**********
Trace: index_construction
    |_CBEventType.NODE_PARSING -> 0.177066 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.008002 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.004272 seconds
      |_CBEventType.CHUNKING -> 0.000269 seconds
      |_CBEventType.CHUNKING -> 0.003683 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.0 seconds
      |_CBEventType.CHUNKING -> 0.006462 seconds
      |

Creating Sub Question Query Engine to Analyze Complex Queries

In [16]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="Kinnaird College Data",
           description="Documents of Collected Data"
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [17]:
response = query_engine.query(
    "Is it worth going into Kinnaird College for bachelors program?"
)
#print(response)
display(Markdown(f"{response}"))

Generated 2 sub questions.
[Kinnaird College Data] Q: What is the acceptance rate at Kinnaird College?
[Kinnaird College Data] Q: What are the average SAT scores of students admitted to Kinnaird College?
[Kinnaird College Data] A: Kinnaird College for Women does not provide specific information regarding the average SAT scores of students admitted. Admission to the college is based on merit criteria specified for different academic programs and qualifications such as Matriculation, 'O' Levels/Equivalent Examinations, and other requirements outlined in the admission policy.
[Kinnaird College Data] A: The acceptance rate at Kinnaird College is not explicitly mentioned in the provided context information.


It can be worth considering Kinnaird College for a bachelor's program due to its merit-based admission criteria and the emphasis on academic qualifications such as Matriculation, 'O' Levels/Equivalent Examinations, and other specified requirements. The decision to attend should be based on personal academic goals, program offerings, campus environment, and other factors that align with individual preferences and aspirations.

In [18]:
response = query_engine.query(
    "What makes Kinnaird different from other Universities? Tell me in Detail of its  uniqueness.")

#print(response)
display(Markdown(f"{response}"))

Generated 2 sub questions.
[Kinnaird College Data] Q: What are the key features of Kinnaird College that set it apart from other universities?
[Kinnaird College Data] Q: Can you provide information on the academic programs offered at Kinnaird College?
[Kinnaird College Data] A: Kinnaird College offers a variety of academic programs including undergraduate and postgraduate programs. The academic programs cover a range of disciplines and are supported by well-equipped science and computer laboratories. Additionally, the campus provides access to the Internet and the Higher Education Commission digital library to support the academic programs.
[Kinnaird College Data] A: Kinnaird College is distinguished by its strong heritage, traditions, and values, along with a commitment to innovation and improvement. The institution emphasizes quality teaching and learning, offers a rich array of extra-curricular activities, and promotes community service. Additionally, Kinnaird College fosters a cult

Kinnaird College stands out from other universities due to its strong emphasis on heritage, traditions, and values, coupled with a dedication to innovation and advancement. The institution is known for its commitment to quality teaching and learning, a diverse range of extra-curricular activities, and a focus on community service. Moreover, Kinnaird College nurtures a culturally diverse and inclusive environment that encourages students to explore their talents, interests, and contribute positively to their own lives and the community.

In [19]:
response = query_engine.query(
    "How do I pay my fees?"
)
#print(response)
display(Markdown(f"{response}"))

Generated 1 sub questions.
[Kinnaird College Data] Q: What are the payment methods available for fees at Kinnaird College?
[Kinnaird College Data] A: Payment methods available for fees at Kinnaird College include cash payments and bank drafts.


You can pay your fees at Kinnaird College using cash payments or bank drafts.

In [20]:
response = query_engine.query(
    "Tell me about the faculty"
)
#print(response)
display(Markdown(f"{response}"))

Generated 2 sub questions.
[Kinnaird College Data] Q: What are the qualifications of the faculty members at Kinnaird College?
[Kinnaird College Data] Q: How many faculty members are there at Kinnaird College?
[Kinnaird College Data] A: Kinnaird College has more than 80 faculty members certified by the British Council.
[Kinnaird College Data] A: Kinnaird College faculty members are certified by the British Council and work as facilitators for the Active Citizenship Program.


The faculty members at Kinnaird College are certified by the British Council and serve as facilitators for the Active Citizenship Program.

In [21]:
response = query_engine.query(
    "Give me a layout of college map?"
)
#print(response)
display(Markdown(f"{response}"))

Generated 1 sub questions.
[Kinnaird College Data] Q: Can the Kinnaird College Data tool provide a map of the college campus?
[Kinnaird College Data] A: The Kinnaird College Data tool does not provide a map of the college campus.


The Kinnaird College Data tool does not provide a map of the college campus.

In [22]:
response = query_engine.query(
    "Give me an idea about the campus. Tell me in detail"
)
#print(response)
display(Markdown(f"{response}"))

Generated 2 sub questions.
[Kinnaird College Data] Q: What are the facilities available on the campus?
[Kinnaird College Data] Q: What are the academic programs offered at the college?
[Kinnaird College Data] A: The academic programs offered at the college include BS Accounting & Finance, Bachelor of Business Administration, BS Computer Science, BS Economics, BS Mathematics, BS Statistics, BS Chemistry, BS Environmental Sciences, BS Geography, BS Physics, BS Biochemistry, BS Biotechnology, BS Botany, BS Food Science & Human Nutrition, BS Genetics, BS Zoology, BS Applied Linguistics, B.Ed. Education, BS English Language & Literature, BS Urdu Literature, Bachelor of Fine Arts, Bachelor of Design, BS Media Studies, BS Applied Psychology, BS International Relations, BS Political Science, and Bachelor of Laws.
[Kinnaird College Data] A: Residential facilities available on the campus include three hostels with dormitories, bedrooms for triple, double, or single occupancy, individual common r

The campus offers residential facilities with three hostels that have dormitories and bedrooms for triple, double, or single occupancy. Each hostel has individual common rooms, and there is a central dining room for all hostel residents. Additionally, the college provides a wide range of academic programs including BS Accounting & Finance, Bachelor of Business Administration, BS Computer Science, BS Economics, BS Mathematics, BS Statistics, BS Chemistry, BS Environmental Sciences, BS Geography, BS Physics, BS Biochemistry, BS Biotechnology, BS Botany, BS Food Science & Human Nutrition, BS Genetics, BS Zoology, BS Applied Linguistics, B.Ed. Education, BS English Language & Literature, BS Urdu Literature, Bachelor of Fine Arts, Bachelor of Design, BS Media Studies, BS Applied Psychology, BS International Relations, BS Political Science, and Bachelor of Laws.

In [23]:
response = query_engine.query(
    "What is the disiciplinary committee?What if someone violates a rule?"
)
#print(response)
display(Markdown(f"{response}"))

Generated 2 sub questions.
[Kinnaird College Data] Q: What is the disciplinary committee at Kinnaird College?
[Kinnaird College Data] Q: What happens if someone violates a rule at Kinnaird College?
[Kinnaird College Data] A: The consequences for violating a rule at Kinnaird College include disciplinary actions such as being issued a Red Card, facing suspension or expulsion from the college, being awarded a grade F in courses, facing a fine, or being subjected to severe disciplinary actions like suspension or expulsion.
[Kinnaird College Data] A: The disciplinary committee at Kinnaird College is responsible for making decisions and taking disciplinary actions that are final and unchallengeable. The college authorities have the authority and jurisdiction over all students during and after college hours.


The disciplinary committee is responsible for making final and unchallengeable decisions regarding disciplinary actions at Kinnaird College. If someone violates a rule at the college, they may face consequences such as being issued a Red Card, suspension or expulsion, receiving a grade F in courses, being fined, or undergoing severe disciplinary actions like suspension or expulsion.

In [24]:
response = query_engine.query(
    "What is the fees of BSCS?"
)
#print(response)
display(Markdown(f"{response}"))

Generated 1 sub questions.
[Kinnaird College Data] Q: What is the fees structure for BSCS at Kinnaird College?
[Kinnaird College Data] A: The fee structure for BSCS at Kinnaird College includes an Admission Fee of Rs. 2,000, Registration Fee of Rs. 2,000, Library Security (Refundable) of Rs. 5,000, Endowment Fund of Rs. 7,800, Tuition Fee of Rs. 156,000, and Practical Subject Fee of Rs. 10,000.


The fees for BSCS at Kinnaird College include an Admission Fee of Rs. 2,000, Registration Fee of Rs. 2,000, Library Security (Refundable) of Rs. 5,000, Endowment Fund of Rs. 7,800, Tuition Fee of Rs. 156,000, and Practical Subject Fee of Rs. 10,000.

In [25]:
response = query_engine.query(
    "What is the fees for MBA program?"
)
#print(response)
display(Markdown(f"{response}"))

Generated 1 sub questions.
[Kinnaird College Data] Q: What is the fees for MBA program at Kinnaird College?
[Kinnaird College Data] A: The fees for the MBA program at Kinnaird College are as follows:
- Admission Fee: Rs. 2,000
- Registration Fee: Rs. 2,000
- Library Security (Refundable): Rs. 5,000
- Endowment Fund: Rs. 7,800
- Tuition Fee: Rs. 156,000


The fees for the MBA program at Kinnaird College include an Admission Fee, Registration Fee, Library Security (Refundable), Endowment Fund, and Tuition Fee.

In [26]:
response = query_engine.query(
    "Who is Amna Ali?"
)
#print(response)
display(Markdown(f"{response}"))

Generated 1 sub questions.
[Kinnaird College Data] Q: Is there any information about Amna Ali in the Kinnaird College Data?
[Kinnaird College Data] A: There is no information about Amna Ali in the provided context.


There is no information available about Amna Ali in the provided context.

In [27]:
response = query_engine.query(
    "Give me Academic Calendar for Fall semester"
)
#print(response)
display(Markdown(f"{response}"))

Generated 1 sub questions.
[Kinnaird College Data] Q: What is the academic calendar for the Fall semester at Kinnaird College?
[Kinnaird College Data] A: The academic calendar for the Fall semester at Kinnaird College typically includes the start date of classes, important deadlines for registration, add/drop periods, midterm exams, final exams, and the end date of the semester.


The Academic Calendar for the Fall semester at Kinnaird College includes the start date of classes, important deadlines for registration, add/drop periods, midterm exams, final exams, and the end date of the semester.

In [28]:
response = query_engine.query(
    "What is the Academic Calendar for Spring semester?"
)
#print(response)
display(Markdown(f"{response}"))

Generated 1 sub questions.
[Kinnaird College Data] Q: What is the Academic Calendar for Spring semester?
[Kinnaird College Data] A: The Academic Calendar for the Spring semester includes the following key dates:
- Spring Session Begins on January 16, 2023
- Kashmir Day on February 05, 2023
- Pre Board Examinations from March 13 to March 22, 2023
- Last day of Classes on April 21, 2023


The Academic Calendar for the Spring semester includes key dates such as the start of the Spring Session on January 16, 2023, Kashmir Day on February 05, 2023, Pre Board Examinations from March 13 to March 22, 2023, and the last day of Classes on April 21, 2023.

Using BM25 Library for Data Retrieval

In [29]:
%pip install llama-index-retrievers-bm25

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [30]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI

In [31]:
llm = OpenAI(model="gpt-3.5-turbo",temperature=0)
splitter = SentenceSplitter(chunk_size=1024)

In [32]:
index = VectorStoreIndex(
    nodes=pdf_nodes,
    storage_context=storage_context,
)

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [33]:
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [34]:
# We can pass in the index, doctore, or list of nodes to create the retriever
retriever = BM25Retriever.from_defaults(nodes=pdf_nodes, similarity_top_k=2)
#import matplotlib.pyplot as plt
from llama_index.core.response.notebook_utils import display_source_node

# will retrieve context from specific companies
nodes = retriever.retrieve("List the faculty for intermediate and undergraduate program for Computer Science students?")
for node in nodes:
    display_source_node(node)

**Node ID:** fbddb5e1-3a15-4aec-81ec-52b1bd43a36d<br>**Similarity:** 7.405791121375211<br>**Text:** Undergraduate 
ProgramsFaculty of Applied Sciences & Business Studies
BS Accounting & Finance
Bac...<br>

**Node ID:** e4790fac-d32c-49f9-a903-21ad239aa3c6<br>**Similarity:** 7.307204881397468<br>**Text:** To receive a BS / BCS / BBA / B.Ed. degree, a 
student must complete a total of 130 credit hours ...<br>

In [35]:
from llama_index.core.tools import RetrieverTool

vector_retriever = VectorIndexRetriever(pdf_index)
bm25_retriever = BM25Retriever.from_defaults(nodes=pdf_nodes, similarity_top_k=2)

retriever_tools = [
    RetrieverTool.from_defaults(
        retriever=vector_retriever,
        description="Useful in most cases",
    ),
    RetrieverTool.from_defaults(
        retriever=bm25_retriever,
        description="Useful if searching about specific information",
    ),
]

In [36]:
from llama_index.core.retrievers import RouterRetriever

retriever = RouterRetriever.from_defaults(
    retriever_tools=retriever_tools,
    llm=llm,
    select_multi=True,
)

In [37]:
# will retrieve all context 
nodes = retriever.retrieve(
    "Can you give me details regarding the faculty of computer science department?"
)
for node in nodes:
    display_source_node(node)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Selecting retriever 1: The question is specific and requires detailed information about a particular topic, which aligns with the second choice..


**Node ID:** f817f92e-f1e8-4e1c-9aa8-3eaebd30e2a7<br>**Similarity:** 7.053763539363047<br>**Text:** BS Biochemistry
The department strives to instill basic knowledge 
of Biochemistry among the stud...<br>

**Node ID:** be563326-1800-40ae-8bb3-60e386d13748<br>**Similarity:** 7.038727614154405<br>**Text:** Kinnaird Library provides a dynamic and 
interactive learning environment with its 
innovative an...<br>

In [38]:
from llama_index.retrievers.bm25 import BM25Retriever

# retireve the top 10 most similar nodes using embeddings
vector_retriever = index.as_retriever(similarity_top_k=10)

# retireve the top 10 most similar nodes using bm25
bm25_retriever = BM25Retriever.from_defaults(nodes=pdf_nodes, similarity_top_k=10)

In [39]:
from llama_index.core.retrievers import BaseRetriever


class HybridRetriever(BaseRetriever):
    def __init__(self, vector_retriever, bm25_retriever):
        self.vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever
        super().__init__()

    def _retrieve(self, query, **kwargs):
        bm25_nodes = self.bm25_retriever.retrieve(query, **kwargs)
        vector_nodes = self.vector_retriever.retrieve(query, **kwargs)

        # combine the two lists of nodes
        all_nodes = []
        node_ids = set()
        for n in bm25_nodes + vector_nodes:
            if n.node.node_id not in node_ids:
                all_nodes.append(n)
                node_ids.add(n.node.node_id)
        return all_nodes

In [40]:
index.as_retriever(similarity_top_k=3)

hybrid_retriever = HybridRetriever(vector_retriever, bm25_retriever)

Node PostProcessing: to Process Response

In [41]:
%pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [42]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-openai

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [43]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings._embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


C:\Users\PMLS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


2 prompts are loaded, with the keys: ['query', 'text']


In [44]:
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=3
)

C:\Users\PMLS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [45]:
from time import time
query_engine = pdf_index.as_query_engine(
    similarity_top_k=10, node_postprocessors=[rerank]
)

now = time()
response = query_engine.query(
    "What are the career prospects for choosing biochemistry as a major?",
)
print(f"Elapsed: {round(time() - now, 2)}s")

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Elapsed: 4.91s


In [46]:
display(Markdown(f"{response}"))

Biochemistry offers various career opportunities in research organizations, academics, and industries. Graduates in Biochemistry can serve in sectors such as health, agriculture, industry, and education. The field of Biochemistry has led to the development of new technologies in various areas of biological sciences, making it an attractive choice for those interested in pursuing careers related to biochemical mechanisms operating in living systems.

CONTINUE RUNNING

In [47]:
from llama_index.core import QueryBundle
from llama_index.core.schema import NodeWithScore

retrieved_nodes = hybrid_retriever._retrieve(
    "Can you give me details regarding the faculty of computer science department?"
)
reranked_nodes = rerank.postprocess_nodes(
    retrieved_nodes,
    query_bundle=QueryBundle(
        "Can you give me details regarding the faculty of computer science department?"
    ),
)

print("Initial retrieval: ", len(retrieved_nodes), " nodes")
print("Re-ranked retrieval: ", len(reranked_nodes), " nodes")

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Initial retrieval:  19  nodes
Re-ranked retrieval:  3  nodes


In [48]:
for node in reranked_nodes:
    display_source_node(node)

**Node ID:** d7be88fe-b43f-44b6-9cd8-a15d0d88400b<br>**Similarity:** -3.9430174827575684<br>**Text:** • All candidates must appear for the KC 
Graduate Assessment Test.
 • Only short listed candidate...<br>

**Node ID:** 76bd4dd4-eee1-46e2-a98f-7861f0af4a70<br>**Similarity:** -4.210566520690918<br>**Text:** BS Computer Science program offers a wide 
matrix of interesting courses, to help students 
attai...<br>

**Node ID:** f817f92e-f1e8-4e1c-9aa8-3eaebd30e2a7<br>**Similarity:** -6.636141777038574<br>**Text:** BS Biochemistry
The department strives to instill basic knowledge 
of Biochemistry among the stud...<br>

In [49]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever=hybrid_retriever,
    node_postprocessors=[rerank],
    llm=llm,
)

response = query_engine.query(
    "What is the impact of cgpa on a student's record?"
)

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [50]:
display(Markdown(f"{response}"))

The CGPA plays a significant role in a student's record as it determines academic standing, probation, and potential dismissal from the program. Maintaining a minimum CGPA is essential for students to avoid being put on probation or dropped from the program. Additionally, the CGPA is used to assess eligibility for academic awards such as graduation with Honors, Gold Medals, Principal's Honor List, and Merit List.

In [51]:
response = query_engine.query(
    "Tell me about teachers of computer science department. Tell me their names"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The teachers in the Computer Science department are fully qualified and are actively involved in the latest developments in the field.

In [52]:
response = query_engine.query(
    "What is the total fees for BCS program?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The total fees for the BCS program amount to 190,900 PKR.

In [53]:
response = query_engine.query(
    "Is this fee for 4 years or a semester?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The fee mentioned in the context is for a year, not a semester.

In [54]:
response = query_engine.query(
    "What fees was mentioned above?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Admission Fee, Registration Fee, Library Security (Refundable), Endowment Fund, Tuition Fee, Practical Subject Fee, Examination Fee, General Funds, Building Funds, Academic Development Fund, Red Crescent Fund, Lahore Board Registration Fee, I.C.S Fee, Research Fund, Per Course Fee, Research Thesis.

Using Similarity PostProcessor

In [55]:
from llama_index.core.postprocessor import SimilarityPostprocessor

postprocessor = SimilarityPostprocessor(similarity_cutoff=0.75)

postprocessor.postprocess_nodes(reranked_nodes)

[]

In [56]:
response = query_engine.query(
    "What is the student code of conduct?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The student code of conduct includes expectations for students to behave maturely, lawfully, and responsibly. It prohibits engaging in disruptive activities, political activities, demonstrations, smoking, possession of illegal substances, theft, cheating, misbehaving with college guards, misuse of mobile phones, indulging in ostentation, and damaging college property. Students are required to wear their college identity cards at all times, attend physical education classes, and refrain from inviting government officials or public personalities on campus without approval. Violations can lead to disciplinary actions such as counseling, probation, warning, suspension, or expulsion.

Using "Refine" Response Synthesizer for structured response

In [57]:
from llama_index.core.data_structs import Node
from llama_index.core.schema import NodeWithScore
from llama_index.core import get_response_synthesizer

response_synthesizer = get_response_synthesizer(response_mode="refine")

query_engine = index.as_query_engine(response_synthesizer=response_synthesizer)


In [58]:
response = query_engine.query(
    "Can I pay fees in installments?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Yes, you can pay the fees in two installments. The first installment is due at the time of admission, and the second installment is due in January of the following year.

In [59]:
response = query_engine.query(
    "Tell me three scholarships program provided by kinnaird"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


1. Scholarship for MPhil postgraduate admissions
2. Scholarship for Undergraduate Program endowed by the Kinnaird College Alumni Association from New York/New Jersey, USA
3. Another scholarship program provided by Kinnaird College

In [60]:
response = query_engine.query(
    "What is the fees for ICS program"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The fees for the ICS program are as follows:
- Tuition Fee: 103,000
- Funds: 23,400
- Practical Subject Fee: 2,000
- ICS Fee: 6,500
- Total: 134,900

In [61]:
response = query_engine.query(
    "Is this fee annually or for the whole program?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The fee mentioned in the context is annual.

In [62]:
response = query_engine.query(
    "What is the eligibility criterion for Nutrition program for bachelors?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


For the Nutrition program for bachelors, the eligibility criterion is a 4-year BS with a minimum CGPA of 2.50 or 60% marks in Masters after a 2-year BSc. Applicants should have completed their Graduation/Masters in fields such as Environmental Sciences, Environmental Engineering, Pure Sciences, Geography, or any other relevant field. Additionally, applicants must have completed Intermediate with Science.

In [63]:
response = query_engine.query(
    "Who is the principal of Kinnaird?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The principal of Kinnaird is Prof. Dr. Rukhsana David.

In [64]:
response = query_engine.query(
    "What academic programs does Kinnaird offer? List them down"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Arts, Social Sciences, and the Sciences.

In [65]:
response = query_engine.query(
    "What is the eligibility criterion for ICS program?",

)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


To be eligible for the ICS program, applicants must be enrolled as a student for the current academic year, have at least 50% marks, be a high achiever in any sports, and demonstrate financial need as verified by the financial aid department.

In [66]:
response = query_engine.query(
    "What if someone has 464 marks out of 545, will she get admission in ICS Stats?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Yes, someone with 464 marks out of 545 would be eligible for admission in ICS (Statistics) based on the provided criteria, which states that applicants need to have 70% marks in Matriculation for admission to ICS (Statistics).

In [67]:
response = query_engine.query(
    "What is the curriculum for Law"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


To receive a LL.B Honors degree, a student must complete a total of 166 credit hours with a minimum CGPA of 2.50.

In [68]:
response = query_engine.query(
    "What student clubs are available. Tell me about them"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


There are 16 different student clubs available, each closely linked with relevant departments. The clubs include Economic Club, English Club, Entrepreneurial Club, Environmental Science Club, Fine Arts Club, Food Science Club, French Club, Geography Club, History and Political Science Club, International Relations Club, Media Club, Music Club, Philosophy Club, Psychology Club, Science Club, and Urdu Club. These clubs provide students with opportunities to discover and showcase their talents, with activities throughout the year being departmental-oriented. Students can join the annual tutorial, the most influential event of any club or society, regardless of their department.

In [69]:
response = query_engine.query(
    "Who are you?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I am KBot, a chatbot to help you answer all your queries related to Kinnaird College for Women University.

In [70]:
response = query_engine.query(
    "Who is the principal of Kinnaird?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The principal of Kinnaird is Prof. Dr. Rukhsana David.

In [71]:
response = query_engine.query(
    "WHo is the vice principal?"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Prof. Dr. Nikhat Khan

In [72]:
response = query_engine.query(
    "Does Kinnaird participate in competitions for extra curricular activities"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Yes, Kinnaird College participates in competitions for extracurricular activities, as highlighted in the provided information.

In [73]:
response = query_engine.query(
    "What are the Frequently Asked Questions"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The Frequently Asked Questions include queries about the identity of the chatbot, admission opening times for Intermediate and Undergraduate studies, the frequency of admissions, the significance of June admissions, program offerings in spring semester admissions, fee structure, installment payment options, and the conduct of admission tests on campus.

In [74]:
response = query_engine.query(
    "give me answers to the above frequently asked questions"
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


IamKBot.AchatbottohelpyouanswerallyourqueriesrelatedtoKinnairdCollegeforWomenUniversity.AdmissionsforintermediatestudiesstartaftertheBoardofIntermediateandSecondaryEducationreleasesthematriculationresults.StudentswhohavecompletedtheirO-levelsandreceivedtheirCambridgeresultscanalsoapply,buttherequirementsforadmissionmayvaryforthem..AdmissionsforintermediatestudiesstartaftertheBoardofIntermediateandSecondaryEducationreleasestheintermediateresults.StudentswhohavecompletedtheirA-levelsandreceivedtheirCambridgeresultscanalsoapply.Admissionsareopenedtwiceayear.Juneadmissionsarethemainadmissionsandofferavarietyof42differentprograms.No,admissionsforspringareavailableforspecificprogramsonly.ThefeestructurevariesdependingontheprogramandisavailableontheprospectusandonlineontheCMSportal.Yes,feescanbepaidininstallments.Admissiontestsareconductedoncampus.

In [75]:
response = query_engine.query(
    "what is the future of kinnaird "
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Kinnaird aims to continue its mission of educating young women by providing them with intellectual capacity, a sense of purpose, leadership abilities, and a commitment to social responsibility.

In [76]:
response = query_engine.query(
    "When does admissions open for intermediate? "
)

display(Markdown(f"{response}"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Admissions for intermediate studies start after the Board of Intermediate and Secondary Education releases the matriculation results. Students who have completed their O-levels and received their Cambridge results can also apply, but the requirements for admission may vary for them.

In [77]:
# import time


# def read_write_loop(input_file,output_file):
#     while True:
#         with open(input_file,"r") as in_file:
#             text_data=in_file.read()
#         if text_data:
#             with open(output_file,"w") as out_file:
#                 response = query_engine.query(text_data)
#                 out_file.write(str(response))

#             with open(input_file,"w") as in_file:
#                 in_file.write("")
            
#             print(f"Text written to {output_file}. Input file cleared")
#         time.sleep(1)
# input_file="input.txt"
# output_file="output.txt"

# read_write_loop(input_file,output_file)

In [78]:
# %pip install -q flask

In [79]:
# from flask import Flask, request, jsonify
# # from flask_cors import CORS
# from llama_index.core import StorageContext, load_index_from_storage

# app = Flask(__name__)
# # cors = CORS(app, resources={r"/response": {"origins": "*"}})


# @app.route('/response', methods=['POST'])
# def get_response():
#     data = request.json
#     query = data.get("query", "")
#     if not query:
#         return jsonify({"error": "Query is required"}), 400

#     # Load the index and create the query engine inside the route
#     # storage_context = StorageContext.from_defaults(persist_dir="./KBot Storage")
#     # index = load_index_from_storage(storage_context)
#     # query_engine = index.as_query_engine()

#     response = query_engine.query(query)
#     return jsonify({"response": str(response)})

# if __name__ == '__main__':
#     app.run(debug=True)

In [80]:
# Persisting to disk
pdf_index.storage_context.persist(persist_dir="./KBot Storage")


In [81]:
import os
print("Current directory:", os.getcwd())


Current directory: c:\Users\PMLS\Documents\University\FYP\KBot\backend
